In [25]:
import torch
import numpy as np
from torch import nn
from torchvision.models import *

In [26]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
device = "cpu"

In [27]:
model = get_model("efficientnet_v2_s", num_classes=345, dropout=0.0)
model.features[0][0] = nn.Conv2d(1, 24, 3, 2, 1, bias=False)
model.eval()
model.to(device)
state = torch.load("./checkpoints/temp-efficientnet_v2_s-QuickDraw")
model.load_state_dict(state["model"])

<All keys matched successfully>

In [28]:
import torch.onnx

In [55]:
model.eval()
x = torch.randn(256, 1, 28, 28, requires_grad=True).to(device)
torch_out = model(x)
# torch.onnx.export(model, x, "efficientnet_v2_s_quickdraw.onnx", export_params=True, do_constant_folding=True, input_names = ["input"], output_names = ["output"], dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})
# onnx_program = torch.onnx.dynamo_export(model, x)

In [25]:
onnx_program.save("effnetv2squickdraw.onnx")

In [1]:
import onnx

In [33]:
onnx_model = onnx.load("efficientnet_v2_s_quickdraw.onnx")
onnx.checker.check_model(onnx_model)

In [34]:
import onnxruntime

In [35]:
import numpy as np

In [36]:
x = np.random.rand(1, 1, 28, 28).astype("float32")

In [56]:
onnx_input = onnx_program.adapt_torch_inputs_to_onnx(x)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

Input length: 1
Sample input: (tensor([[[[ 1.7735e+00,  1.0513e-01,  1.0396e+00,  ...,  9.8241e-01,
            3.5688e-01, -1.1140e+00],
          [-1.0462e+00,  2.4896e-01,  2.0069e+00,  ...,  2.3443e-01,
            8.1108e-01,  4.2251e-01],
          [ 2.2605e+00,  8.0056e-01, -6.2645e-01,  ..., -7.6775e-01,
           -1.9519e+00, -1.0537e+00],
          ...,
          [-2.2383e+00, -6.9262e-01,  1.6229e-01,  ...,  6.5564e-01,
            1.8345e-01, -1.5152e+00],
          [-2.8821e-01, -7.1788e-02,  1.4528e+00,  ..., -7.0096e-01,
            8.8242e-01,  2.3232e-01],
          [-2.0215e-01, -5.9241e-01, -1.9561e+00,  ..., -7.9916e-01,
           -1.0904e+00, -4.7240e-01]]],


        [[[-5.0371e-01, -4.7335e-01,  1.6114e+00,  ..., -5.9548e-02,
            7.4131e-02,  5.9113e-01],
          [ 6.4886e-01, -1.4591e+00,  1.1717e+00,  ...,  2.6022e-01,
           -1.1192e+00, -8.4942e-01],
          [ 2.1373e-01,  6.2774e-01, -1.6612e+00,  ..., -7.8231e-01,
           -7.9880e-01,  

In [49]:
ort_session = onnxruntime.InferenceSession("efficientnet_v2_s_quickdraw.onnx", providers=["CUDAExecutionProvider"])

In [39]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [57]:
onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

In [40]:
ort_session.get_inputs()[0].name

'input'

In [58]:
onnxruntime_input = {"input": x}
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

RuntimeError: Input must be a list of dictionaries or a single numpy array for input 'input'.

In [59]:
torch_outputs = model(x)
torch_outputs = onnx_program.adapt_torch_outputs_to_onnx(torch_outputs)

In [62]:
assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output.to("cpu"), torch.tensor(onnxruntime_output), rtol=1e-3, atol=1e-5)

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

AssertionError: Tensor-likes are not close!

Mismatched elements: 4 / 88320 (0.0%)
Greatest absolute difference: 2.944469451904297e-05 at index (217, 5) (up to 1e-05 allowed)
Greatest relative difference: 0.01649314910173416 at index (208, 152) (up to 0.001 allowed)

In [63]:
import time

device = "cuda"
model.to(device)
x = torch.randn(256, 1, 28, 28, requires_grad=True).to(device)

start = time.time()
torch_out = model(x)
end = time.time()
print(f"Inference of Pytorch model used {end - start} seconds")

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
start = time.time()
ort_outs = ort_session.run(None, ort_inputs)
end = time.time()
print(f"Inference of ONNX model used {end - start} seconds")

Inference of Pytorch model used 0.46076011657714844 seconds
Inference of ONNX model used 0.11152815818786621 seconds


In [53]:
ort_outs

[array([[ -1.7509418 ,  -7.0257072 ,  -6.1707773 , ..., -16.943882  ,
         -10.683023  ,   1.3855481 ],
        [ -0.64571816,  -3.9192946 ,  -4.384699  , ...,  -8.295189  ,
          -7.0409455 ,   1.6498849 ],
        [ -2.6629148 ,  -9.015192  ,  -4.5159264 , ..., -19.517633  ,
         -13.337657  ,   0.9483351 ],
        ...,
        [-10.925127  , -18.034628  , -17.231544  , ..., -33.50666   ,
         -17.958172  ,  -4.6431284 ],
        [ -4.5861526 , -11.174621  ,  -9.48193   , ..., -16.354448  ,
          -9.117007  ,   3.5767696 ],
        [ -5.615005  , -11.875569  , -11.360513  , ..., -17.924488  ,
          -8.923729  ,   3.0314457 ]], dtype=float32)]

In [64]:
torch.testing.assert_close(to_numpy(torch_out), ort_outs[0], rtol=1e-3, atol=1e-5)

AssertionError: Tensor-likes are not close!

Mismatched elements: 33052 / 88320 (37.4%)
Greatest absolute difference: 0.11327171325683594 at index (59, 342) (up to 1e-05 allowed)
Greatest relative difference: 44.576210021972656 at index (72, 86) (up to 0.001 allowed)

In [65]:
for torch_output, onnxruntime_output in zip(torch_out, ort_outs[0]):
    torch.testing.assert_close(torch_output.to("cpu"), torch.tensor(onnxruntime_output), rtol=1e-3, atol=1e-5)

AssertionError: Tensor-likes are not close!

Mismatched elements: 91 / 345 (26.4%)
Greatest absolute difference: 0.021764755249023438 at index (204,) (up to 1e-05 allowed)
Greatest relative difference: 0.1622679978609085 at index (190,) (up to 0.001 allowed)

In [66]:
onnxruntime.get_device()

'GPU'

In [ ]:
%env ONNX_MODE=cuda

env: ONNX_MODE=cuda


In [67]:
print(onnxruntime.get_available_providers())

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [68]:
import QuickDraw as Data

test = Data.test_dataloader(256)


################################
########## refreshing ##########
################################


In [60]:
print(ort_session.get_providers())

['CPUExecutionProvider']


In [69]:
correct = 0
total = 0

for images, labels in test:
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(images)}
    ort_outs = ort_session.run(None, ort_inputs)
    preds = np.argmax(ort_outs[0], axis=1)
    correct += (preds == labels.numpy()).sum()
    total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 76.77%


In [70]:
correct = 0
total = 0
model.eval()
for images, labels in test:
    images, labels = images.to(device), labels.to(device)
    
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(images)
    
    _, preds = torch.max(outputs, 1)
    correct += (preds == labels).sum().item()
    total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 76.77%
